In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D as KerasConv2D, MaxPooling2D as KerasMaxPooling2D, Flatten as KerasFlatten, Dropout as KerasDropout
from tensorflow.keras.models import Model

# Assuming your custom layers are in a 'layers' directory relative to your notebook
# Make sure the paths in your uploaded files (e.g., from .layer import Layer) are resolvable
# For example, if 'layers' is a package, it might need an __init__.py
# For simplicity here, we'll assume the .py files are directly accessible in a 'layers' folder.

# Import your custom layers
from layers.conv2d import Conv2D as CustomConv2D # Assuming you have this file
from layers.pooling import MaxPooling2D as CustomMaxPooling2D
from layers.flatten import Flatten as CustomFlatten
from layers.dropout import Dropout as CustomDropout

# Helper to print comparison results
def print_outputs_and_comparison(keras_out, custom_out, layer_name):
    print(f"\n--- {layer_name} ---")
    print(f"Keras output shape: {keras_out.shape}")
    print(f"Custom output shape: {custom_out.shape}")
    print("Keras output sample:\n", keras_out.flatten()[:8])
    print("Custom output sample:\n", custom_out.flatten()[:8])
    if keras_out.shape == custom_out.shape and np.allclose(keras_out, custom_out, atol=1e-6):
        print(f"✅ Outputs for {layer_name} are identical or very close!")
    else:
        print(f"⚠️ Outputs for {layer_name} are DIFFERENT.")
        diff = np.abs(keras_out - custom_out)
        print(f"   Max absolute difference: {np.max(diff)}")
        # print(f"   Mean absolute difference: {np.mean(diff)}")

In [19]:
import importlib
import sys

# --- Modules to Reload ---
module_paths = {
    "LayerBase": "layers.layer",
    "Conv2D": "layers.conv2d",
    "Sequential": "sequential",
    "MaxPooling2D": "layers.pooling",
    "Flatten": "layers.flatten",
    "Dense": "layers.dense",
    "Dropout": "layers.dropout",
}

print("--- Attempting to Reload Modules ---")
for logical_name, module_path_str in module_paths.items():
    if module_path_str in sys.modules:
        try:
            # Get the actual module object from sys.modules
            module_to_reload = sys.modules[module_path_str]
            importlib.reload(module_to_reload)
            print(f"Successfully reloaded: {module_path_str} (as {logical_name})")
        except Exception as e:
            print(f"Error reloading {module_path_str}: {e}")
    else:
        print(f"Module {module_path_str} not in sys.modules. Will attempt fresh import.")


--- Attempting to Reload Modules ---
Successfully reloaded: layers.layer (as LayerBase)
Successfully reloaded: layers.conv2d (as Conv2D)
Module sequential not in sys.modules. Will attempt fresh import.
Successfully reloaded: layers.pooling (as MaxPooling2D)
Successfully reloaded: layers.flatten (as Flatten)
Module layers.dense not in sys.modules. Will attempt fresh import.
Successfully reloaded: layers.dropout (as Dropout)


In [20]:
# Input data configuration
batch_size = 2
img_height = 28
img_width = 28
channels = 3 # Using 3 channels for a more general Conv2D test

# Generate random input data
# Adding a high value to ensure weights have a noticeable effect
sample_input_data = np.random.rand(batch_size, img_height, img_width, channels).astype(np.float32) * 10

print(f"Sample input data shape: {sample_input_data.shape}")

Sample input data shape: (2, 28, 28, 3)


In [21]:
# Keras Model
keras_input = Input(shape=(img_height, img_width, channels))
keras_conv = KerasConv2D(filters=4, kernel_size=(3, 3), padding='valid', activation='relu', name="conv2d_1")(keras_input)
keras_pool = KerasMaxPooling2D(pool_size=(2, 2), name="maxpool_1")(keras_conv)
keras_flatten = KerasFlatten(name="flatten_1")(keras_pool)
keras_dropout = KerasDropout(rate=0.5, name="dropout_1")(keras_flatten) # Rate doesn't matter for inference
keras_model = Model(inputs=keras_input, outputs=[keras_conv, keras_pool, keras_flatten, keras_dropout])

# Get intermediate outputs from Keras model for comparison
keras_conv_out_model = Model(inputs=keras_input, outputs=keras_conv)
keras_pool_out_model = Model(inputs=keras_input, outputs=keras_pool)
keras_flatten_out_model = Model(inputs=keras_input, outputs=keras_flatten)
keras_dropout_out_model = Model(inputs=keras_input, outputs=keras_dropout)


# Custom Layers Instantiation
# Ensure parameters match Keras layers
custom_conv = CustomConv2D(filters=4, kernel_size=(3,3), padding='valid', activation='relu')
custom_pool = CustomMaxPooling2D(pool_size=(2, 2), padding='valid') # Assuming 'valid' is default or handled
custom_flatten = CustomFlatten()
custom_dropout = CustomDropout() # Dropout rate is irrelevant for inference

print("Keras model and custom layers initialized.")

Keras model and custom layers initialized.


In [22]:
# It's good practice to build the model or run a dummy input to ensure weights are created
_ = keras_model.predict(sample_input_data)

# --- Conv2D ---
keras_conv_layer = keras_model.get_layer("conv2d_1")
conv_weights = keras_conv_layer.get_weights() # [kernel, bias]
if hasattr(custom_conv, 'load_keras_weights'):
    # Assuming CustomConv2D.load_keras_weights expects [kernel, bias]
    custom_conv.load_keras_weights(conv_weights)
    print(f"Loaded weights for Conv2D. Kernel shape: {conv_weights[0].shape}, Bias shape: {conv_weights[1].shape}")
else:
    print("⚠️ CustomConv2D does not have load_keras_weights method. Skipping weight loading for Conv2D.")


# --- MaxPooling2D ---
# MaxPooling2D has no trainable weights
if hasattr(custom_pool, 'load_keras_weights'):
    custom_pool.load_keras_weights([]) # Pass empty list or handle accordingly
    print("Called load_keras_weights for MaxPooling2D (no weights).")
else:
    # This is fine as per your pooling.py which doesn't have load_keras_weights
    print("CustomMaxPooling2D does not have load_keras_weights method (expected for pooling).")


# --- Flatten ---
# Flatten has no trainable weights
if hasattr(custom_flatten, 'load_keras_weights'):
    custom_flatten.load_keras_weights([])
    print("Called load_keras_weights for Flatten (no weights).")
else:
    # This is fine as per your flatten.py which does have it
    print("⚠️ CustomFlatten does not have load_keras_weights method. Ensure this is intended.")


# --- Dropout ---
# Dropout has no trainable weights
if hasattr(custom_dropout, 'load_keras_weights'):
    custom_dropout.load_keras_weights([])
    print("Called load_keras_weights for Dropout (no weights).")
else:
    # This is fine as per your dropout.py which does have it
    print("⚠️ CustomDropout does not have load_keras_weights method. Ensure this is intended.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
Loaded weights for Conv2D. Kernel shape: (3, 3, 3, 4), Bias shape: (4,)
Pooling has no trainable weights — skipping
Called load_keras_weights for MaxPooling2D (no weights).
Flatten has no trainable weights — skipping
Called load_keras_weights for Flatten (no weights).
Dropout has no trainable weights — skipping
Called load_keras_weights for Dropout (no weights).


In [23]:
# --- Test Conv2D Layer ---
try:
    # Keras prediction
    keras_conv_output = keras_conv_out_model.predict(sample_input_data)

    # Custom prediction
    # Assuming CustomConv2D has a forward method
    custom_conv_output = custom_conv.forward(sample_input_data)
    print_outputs_and_comparison(keras_conv_output, custom_conv_output, "Conv2D")
except Exception as e:
    print(f"Error during Conv2D test: {e}")
    keras_conv_output = None # Ensure variable exists for next step
    custom_conv_output = None


# --- Test MaxPooling2D Layer ---
if custom_conv_output is not None: # Proceed only if previous layer output is valid
    try:
        # Keras prediction (output from previous Keras layer is input to next)
        keras_pool_output = keras_pool_out_model.predict(sample_input_data) # Gets output after conv and pool

        # Custom prediction (output from custom_conv is input here)
        custom_pool_output = custom_pool.forward(custom_conv_output)
        print_outputs_and_comparison(keras_pool_output, custom_pool_output, "MaxPooling2D")
    except Exception as e:
        print(f"Error during MaxPooling2D test: {e}")
        keras_pool_output = None
        custom_pool_output = None
else:
    print("\nSkipping MaxPooling2D test due to previous layer failure.")
    keras_pool_output = None
    custom_pool_output = None


# --- Test Flatten Layer ---
if custom_pool_output is not None: # Proceed only if previous layer output is valid
    try:
        # Keras prediction
        keras_flatten_output = keras_flatten_out_model.predict(sample_input_data)

        # Custom prediction
        custom_flatten_output = custom_flatten.forward(custom_pool_output)
        print_outputs_and_comparison(keras_flatten_output, custom_flatten_output, "Flatten")
    except Exception as e:
        print(f"Error during Flatten test: {e}")
        keras_flatten_output = None
        custom_flatten_output = None
else:
    print("\nSkipping Flatten test due to previous layer failure.")
    keras_flatten_output = None
    custom_flatten_output = None

# --- Test Dropout Layer ---
if custom_flatten_output is not None: # Proceed only if previous layer output is valid
    try:
        # Keras prediction
        # Note: Keras Dropout layer behaves as identity during inference (model.predict)
        keras_dropout_output = keras_dropout_out_model.predict(sample_input_data)

        # Custom prediction
        # Your custom Dropout.forward(x) returns x, which is correct for inference
        custom_dropout_output = custom_dropout.forward(custom_flatten_output)
        print_outputs_and_comparison(keras_dropout_output, custom_dropout_output, "Dropout")
    except Exception as e:
        print(f"Error during Dropout test: {e}")
else:
    print("\nSkipping Dropout test due to previous layer failure.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step

--- Conv2D ---
Keras output shape: (2, 26, 26, 4)
Custom output shape: (2, 26, 26, 4)
Keras output sample:
 [1.1282748 0.        2.7675052 0.        2.5542336 0.        0.
 0.       ]
Custom output sample:
 [1.12827468 0.         2.76750469 0.         2.55423355 0.
 0.         0.        ]
✅ Outputs for Conv2D are identical or very close!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step

--- MaxPooling2D ---
Keras output shape: (2, 13, 13, 4)
Custom output shape: (2, 13, 13, 4)
Keras output sample:
 [3.1308131  0.         2.7675052  0.25489885 1.7260418  0.
 3.969039   2.1871903 ]
Custom output sample:
 [3.13081288 0.         2.76750469 0.25489867 1.72604144 0.
 3.9690392  2.18719006]
✅ Outputs for MaxPooling2D are identical or very close!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step

--- Flatten ---
Keras output shape: (2, 676)
Custom output shape: (2, 676)
Keras output sample:
 [3.1308131  0.         2.7675052  0.25489885 1.7260418  0.
 3.969039   2.1871903 ]
C